# Understand Closure and Decorator

## 函数作为参数和返回值  
* 高阶函数(high-order function)  
把函数作为参数或者是返回值的函数

## 变量的作用域  
* LEGB 规则
    * local
    * enclosed
    * global
    * built-in

In [ ]:
# 通过一个例子来看变量的作用域的范围

# 定义一个全局变量
GLOBAL_VAR = 'G'

# 定义一个函数, 此函数嵌套了一个子函数
def func1():
    
    # 这个变量是外层函数拥有, 但是内层函数有调用, 它被称为Enclosed
    enclosure_var = 'e'
    
    # 内层函数和外层函数有同名变量, 会优先local的变量
    local_var = 'e'
    def func2():
        local_var ='l'
        # print各变量值 int为 built-in 
        print(f'The vars are: {local_var}, {enclosure_var}, {GLOBAL_VAR}, {int}')
    return func2

func1()()

In [ ]:
# 探讨一个例子, 它的结果是什么? 
def func3():
    enclosure_var = 'e'
    local_var = 'e'
    def func4():
        print(local_var)
        local_var = 'l'
    return func4

In [ ]:
func3()()

In [ ]:
# 另外一个例子
def func5():
    enclosure_list = []
    def func6():
        print(enclosure_list)
        enclosure_list.extend([1,2,3])
    return func6

In [ ]:
func5()()

## 闭包 Closure
* 函数 + 自由变量
* 延伸了作用域的"函数"

In [ ]:
# 一般函数的生命周期在执行完之后就会结束, 在函数内的变量会随着函数栈的返回而销毁. 
# 有没有办法让函数保留状态呢? 
# 闭包是一种实现的方式

# 我们有一个需求, 需要有一个计数器函数, 每调用一次, 它的累计就+1 
def counter_closure():
    count = 0
    def counter():
        nonlocal count
        other_arg = 0
        count += 1
        return count
    return counter

In [ ]:
counter = counter_closure()

In [ ]:
counter()

In [ ]:
counter.__closure__

In [ ]:
counter.__code__.co_freevars

In [ ]:
counter.__code__.co_varnames

In [ ]:
counter_closure.__closure__

In [ ]:
import dis
dis.dis('counter = counter_closure()')

In [ ]:
dis.dis('counter()')

In [ ]:
# 用类来实现counter 
class Counter():
    def __init__(self):
        self.count = 0 

    def __call__(self):
        self.count += 1
        return self.count

counter_c = Counter()
counter_c()

In [ ]:
counter_c()

## 装饰器 Decorator
* 不入侵原代码, 增加或修改功能

In [ ]:
# 举例说明
# 我们现在有一个需求, 想知道一个函数的运行时间是多少. 一个非常粗糙的想法是 
from datetime import datetime
import time
#定义这个函数
def simple_func():
    print(f'running function simple_func')
    time.sleep(0.1)

# 写代码实现新的想法
# 先记录开始时间
start_time = datetime.now()
#执行函数
simple_func()
# 记录结束时间
end_time = datetime.now()

#打印出花的时间
print(f'The function ran for {end_time - start_time}')

In [ ]:
# 我们现在进化一下, 把这个功能编程一个函数

def cal_time_origin():
    
    start_time = datetime.now()
    #执行希望被包装的函数, 同时保留函数的返回值
    r = simple_func()
    # 记录结束时间
    end_time = datetime.now()
    
    #打印出花的时间
    print(f'The function ran for {end_time - start_time}')
    # 返回被包装的函数的返回值
    return r

In [ ]:
cal_time_origin()

In [ ]:
# 比之前的好, 但是它有个问题, 因为 simple_func是hard code在里面. 不能用于更多的函数, 扩展性很差.
# 我们思考一下, 如果把cal_time里面的 simple_func改成变量, 也就是说, 允许cal_time包装任何的函数, 那么这个功能的意义将大大的扩展
# 我们考虑将需要包装的函数名当作cal_time的参数传入, 在cal_time里面执行被包装的函数
# 这里我们使用一个两层的嵌套函数,  内层函数的调用参数用于接收被包装函数的参数, 外层函数的调用参数用于接收被包装的函数名

In [ ]:
# 内层函数就是刚刚介绍的闭包, 外层的参数func是闭包的自由变量, 内层函数里将调用它, 完成被包装函数的执行
def cal_time(func):
    
    # 因为我们无法提前预知被包装函数的参数, 所以我们用不定参数来接收 (*args, **kw), 它的作用是原封不动的把调用参数传递给被包装的函数
    def wrap(*args, **kw):
        start_time = datetime.now()
        r = func(*args, **kw)
        end_time = datetime.now()
        print(f'The function ran for {end_time - start_time}')
        return r
    return wrap

In [ ]:
# 新写另一个函数, 试验是否可以cal_time计算时间函数的执行时间!
import time
def dummy_wait():
    print(f'running function dummy_wait')
    time.sleep(0.1)

In [ ]:
# 调用刚刚写好的计算时间函数, 把需要包装的dummy_wait函数名传入
enclosure_func = cal_time(dummy_wait)

In [ ]:
# 得到了闭包的对象, 再调用它, 实现功能
enclosure_func()

In [ ]:
enclosure_func.__closure__

In [ ]:
enclosure_func.__code__.co_freevars

In [ ]:
# 这个功能很好, 但是实现起来显得不那么优雅, 于是python 发明了一个语法糖, 它就是装饰器
# 同样的效果可以这么写

@cal_time
def dummy_wait2():
    print(f'running function dummy_wait2')
    time.sleep(0.1)

In [ ]:
#这个实现, 形式简洁并且保留了原来函数的执行
dummy_wait2()

In [ ]:
# 通过@cal_time 语法,实际上做的是把被装饰的函数当作参数传入到了装饰器当中
# 以下语法
@cal_time
def func():
    pass

# 做的就是以下的事情:
func = cal_time(func)

# 在被装饰了之后, 函数func, 等于了cal_time(func), 它的执行结果就是闭包wrap
# 之后每次执行func(*args, **kw) 就是执行 wrap(*args, **kw)
# 在wrap里面, 我们实现了不改动原本func的代码, 在执行func函数的同时,新增了功能

In [ ]:
# 目前装饰器的缺点
# 不支持参数
# 实际上函数被替换成了装饰器的闭包函数

from functools import wraps

def cal_time(func):
    
    #functools 里面的wraps装饰器可以用于装饰返回的闭包, 它会把原函数的相关属性复制给闭包
    @wraps(func)
    def wrap(*args, **kw):
        start_time = datetime.now()
        r = func(*args, **kw)
        end_time = datetime.now()
        print(f'The function ran for {end_time - start_time}')
        return r
    return wrap

@cal_time
def func():
    pass

func

In [ ]:
# 带参数的装饰器
# 在原有的装饰器基础上, 再嵌套一层函数, 用于接收装饰器的参数即可

def cal_time_with_comment(comment):
    def deco(func):
        @wraps(func)
        def wrap(*args, **kw):
            print(f'This is the comment {comment}')
            start_time = datetime.now()
            r = func(*args, **kw)
            end_time = datetime.now()
            print(f'The function ran for {end_time - start_time}')
            return r
        return wrap
    return deco

# 在使用装饰器的时候, 直接调用最外层的函数并传入参数, 它的返回, 就是我们之前将解的无参数的装饰器函数. 
# 此后和之前讲过的内容除了增加了新的参数, 其他并无两样
@cal_time_with_comment('Hello Decorator')
def dummy_wait3():
    print(f'running function dummy_wait3')
    time.sleep(0.1)

dummy_wait3()

In [ ]:
# 装饰器的嵌套 

def a(func):
    pass

def b(func):
    pass

@a
@b
def func():
    pass

# 它等同于什么? 